In [ ]:
import os
import ipywidgets as widgets
from IPython.display import display, clear_output

# Variable global para las rutas
rutas_archivos = {}

def preparar_uploader(dest_dir, nombres_base):
    """
    Genera widgets de carga de archivos individuales que detectan automáticamente 
    la extensión del archivo subido y lo guardan con el nombre base especificado.
    
    Args:
        dest_dir (str): Ruta del directorio destino donde se guardarán los archivos.
            Si no existe, se creará automáticamente.
        nombres_base (list): Lista de strings con los nombres base para cada archivo
            (sin extensión). Cada elemento generará un widget de carga independiente.
    
    Returns:
        None: La función no retorna valores. Los resultados se almacenan en la 
            variable global 'rutas_archivos'.
    
    Raises:
        OSError: Cuando no se pueden crear los directorios o escribir archivos.
        PermissionError: Cuando no hay permisos suficientes en el directorio destino.
        IOError: Cuando ocurre un error durante la escritura del archivo.
    
    Notes:
        - La variable global 'rutas_archivos' se limpia al inicio de cada ejecución.
        - Si ya existe un archivo con el mismo nombre en el destino, se sobrescribe.
        - La extensión se detecta automáticamente del archivo original subido.
        - Cada upload es independiente y se puede realizar en cualquier orden.
    
    Examples:
        >>> preparar_uploader("/datos/procesamiento/", ["master", "configuracion"])
        # Crea 2 widgets de carga
        # Si subes archivo.csv al primer widget: rutas_archivos['master'] = '/datos/procesamiento/master.csv'
        # Si subes config.xlsx al segundo: rutas_archivos['configuracion'] = '/datos/procesamiento/configuracion.xlsx'
        
        >>> # Verificar rutas después de cargar
        >>> print(rutas_archivos)
        {'master': '/datos/procesamiento/master.csv', 'configuracion': '/datos/procesamiento/configuracion.xlsx'}
    """
    global rutas_archivos
    
    if not nombres_base:
        print("⚠️ Lista vacía")
        return

    os.makedirs(dest_dir, exist_ok=True)
    rutas_archivos.clear()
    elementos = []

    for idx, nombre_base in enumerate(nombres_base):
        label = widgets.Label(f"🗂️ {idx+1}. Cargar '{nombre_base}'")
        uploader = widgets.FileUpload(accept='*', multiple=False)
        output = widgets.Output()

        def on_upload(change, nombre_base=nombre_base, output=output):
            with output:
                clear_output()
                print(f"🔍 Callback ejecutado para {nombre_base}")
                try:
                    # Usar directamente el change en lugar de uploader.value
                    archivos = change['new']
                    # print(f"🔍 Archivos desde change: {type(archivos)}")
                    # print(f"🔍 Contenido change: {archivos}")
                    
                    if not archivos:
                        print("❌ No hay archivos en change")
                        return
                    
                    # Probar diferentes formas de acceder a los datos
                    if isinstance(archivos, dict):
                        # print("🔍 Es diccionario, obteniendo primer valor")
                        fileinfo = list(archivos.values())[0]
                    elif isinstance(archivos, (list, tuple)) and len(archivos) > 0:
                        # print("🔍 Es lista/tupla, obteniendo primer elemento")
                        fileinfo = archivos[0]
                    else:
                        print(f"❌ Tipo de archivo no reconocido: {type(archivos)}")
                        return
                        
                    # print(f"🔍 FileInfo: {type(fileinfo)}")
                    # print(f"🔍 FileInfo keys: {fileinfo.keys() if hasattr(fileinfo, 'keys') else 'No keys'}")
                    
                    nombre_original = fileinfo['name']
                    contenido = fileinfo['content']
                    # print(f"🔍 Archivo: {nombre_original}, Tamaño: {len(contenido)} bytes")
                    
                    # Detectar extensión
                    extension = os.path.splitext(nombre_original)[1]
                    nombre_final = f"{nombre_base}{extension}"
                    ruta_completa = os.path.join(dest_dir, nombre_final)
                    # print(f"🔍 Ruta final: {ruta_completa}")
                    
                    # Crear directorio si no existe
                    # print(f"🔍 Creando directorio: {dest_dir}")
                    os.makedirs(dest_dir, exist_ok=True)
                    # print(f"🔍 Directorio existe? {os.path.exists(dest_dir)}")
                    
                    # Guardar archivo
                    # print(f"🔍 Escribiendo archivo...")
                    with open(ruta_completa, 'wb') as f:
                        f.write(contenido)
                    # print(f"🔍 Archivo existe? {os.path.exists(ruta_completa)}")
                    
                    # Actualizar diccionario
                    rutas_archivos[nombre_base] = ruta_completa
                    # print(f"🔍 Diccionario actualizado: {rutas_archivos}")
                    
                    print(f"✅ {nombre_original} → {nombre_final}")
                    
                except Exception as e:
                    print(f"❌ Error completo: {type(e).__name__}: {str(e)}")
                    import traceback
                    print(f"🔍 Traceback: {traceback.format_exc()}")

        uploader.observe(on_upload, names='value')
        elementos.append(widgets.VBox([label, uploader, output]))

    display(widgets.VBox(elementos))
    print(f"📁 Guardando en: {dest_dir}")

In [ ]:
preparar_uploader("./data/", ["medicametos_vencidos", "medicamentos_vigentes", "medicamentos_renovacion","medicamentos_otros"])
# Después de cargar:
print(rutas_archivos) 

In [ ]:
rutas_archivos = {'medicametos_vencidos': './data/medicametos_vencidos.xlsx', 'medicamentos_vigentes': './data/medicamentos_vigentes.xlsx', 'medicamentos_renovacion': './data/medicamentos_renovacion.xlsx', 'medicamentos_otros': './data/medicamentos_otros.xlsx'}

print("Rutas de archivos cargados:")
for nombre, ruta in rutas_archivos.items():
    print(f"{nombre}: {ruta}")

In [ ]:
print(rutas_archivos) 

In [ ]:

import polars as pl

df_medicametos_vencidos = pl.read_excel(rutas_archivos['medicametos_vencidos'])
df_medicamentos_vigentes = pl.read_excel(rutas_archivos['medicamentos_vigentes'])
df_medicamentos_renovacion = pl.read_excel(rutas_archivos['medicamentos_renovacion'])
df_medicamentos_otros = pl.read_excel(rutas_archivos['medicamentos_otros'])


In [ ]:
# Imprime las columnas de cada DataFrame separadas por '|'
for nombre, df in [
    ("df_medicametos_vencidos", df_medicametos_vencidos),
    ("df_medicamentos_vigentes", df_medicamentos_vigentes),
    ("df_medicamentos_renovacion", df_medicamentos_renovacion),
    ("df_medicamentos_otros", df_medicamentos_otros)
]:
    print(f"{nombre}:")
    print(" | ".join(df.columns))
    print("-" * 80)

# Verificar si todas las columnas son iguales
columnas_iguales = (
    (df_medicametos_vencidos.columns == df_medicamentos_vigentes.columns == 
        df_medicamentos_renovacion.columns == df_medicamentos_otros.columns)
)
print("¿Todas las columnas son iguales?:", columnas_iguales)

# Verificar si todos los tipos de datos son iguales
tipos_iguales = (
    (df_medicametos_vencidos.dtypes == df_medicamentos_vigentes.dtypes ==
        df_medicamentos_renovacion.dtypes == df_medicamentos_otros.dtypes)
)

print("¿Todos los tipos de datos son iguales?:", tipos_iguales)

In [ ]:
if columnas_iguales and tipos_iguales:
    # Añadir columna 'DATASET' a cada DataFrame con el nombre de la fuente
    df_medicametos_vencidos = df_medicametos_vencidos.with_columns(
        pl.lit("medicametos_vencidos").alias("DATASET")
    )
    df_medicamentos_vigentes = df_medicamentos_vigentes.with_columns(
        pl.lit("medicamentos_vigentes").alias("DATASET")
    )
    df_medicamentos_renovacion = df_medicamentos_renovacion.with_columns(
        pl.lit("medicamentos_renovacion").alias("DATASET")
    )
    df_medicamentos_otros = df_medicamentos_otros.with_columns(
        pl.lit("medicamentos_otros").alias("DATASET")
    )

    # Unir todos los DataFrames
    df_unido = pl.concat([
        df_medicametos_vencidos,
        df_medicamentos_vigentes,
        df_medicamentos_renovacion,
        df_medicamentos_otros
    ])
    print("DataFrames unidos correctamente.")
else:
    print("Las columnas o los tipos de datos no son iguales. No se puede unir.")
    
# Crear la columna "CUM" concatenando EXPEDIENTE CUM y CONSECUTIVO como string
df_unido = df_unido.with_columns(
    (pl.col("EXPEDIENTE CUM").cast(pl.Utf8) + pl.lit("-") + pl.col("CONSECUTIVO").cast(pl.Utf8)).alias("CUM"),
    (pl.col("EXPEDIENTE CUM").cast(pl.Utf8) + pl.lit(".") + pl.col("CONSECUTIVO").cast(pl.Utf8)).cast(pl.Float64).alias("CUM_FLOAT")
)

# Reordenar columnas: poner "CUM" primero y eliminar "EXPEDIENTE", "EXPEDIENTE CUM" y "CONSECUTIVO"
cols = ["CUM","EXPEDIENTE CUM", "CONSECUTIVO"] + [c for c in df_unido.columns if c not in ["CUM", "EXPEDIENTE", "CUM_FLOAT", "EXPEDIENTE CUM", "CONSECUTIVO"]]
df_unido = df_unido.select(cols)

In [ ]:
df_unido

In [ ]:
cols_preproc = ['CUM', 'EXPEDIENTE CUM', "CONSECUTIVO",
 'ESTADO REGISTRO',
 'CANTIDAD CUM',
 'ESTADO CUM',
 'MUESTRA MÉDICA',
 'UNIDAD',
 'ATC',
 'DESCRIPCIÓN_ATC',
 'VÍA ADMINISTRACIÓN',
 'CONCENTRACIÓN',
 'PRINCIPIO ACTIVO',
 'UNIDAD MEDIDA',
 'CANTIDAD',
 'UNIDAD REFERENCIA',
 'FORMA FARMACÉUTICA']

df_preproc = df_unido.select(cols_preproc).sort(['CUM', 'EXPEDIENTE CUM', 'CONSECUTIVO'])
df_preproc

In [ ]:
# Imprimir valores únicos por columna separados por comas
for col in df_preproc.columns:
    valores_unicos = df_preproc[col].unique().to_list()
    # Convertir valores None a string para evitar errores
    valores_str = [str(v) if v is not None else 'None' for v in valores_unicos]
    print(f"{col}:")
    print(", ".join(valores_str))
    print("-" * 80)

In [ ]:
df_preproc

In [ ]:

print("="*60)
print("INFORMACIÓN GENERAL DEL DATASET")
print("="*60)

print(f"📊 Dimensiones: {df_preproc.shape[0]:,} filas × {df_preproc.shape[1]} columnas")
print(f"💾 Memoria utilizada: {df_preproc.estimated_size() / 1024**2:.2f} MB")

print("\n📋 COLUMNAS:")
for i, col in enumerate(df_preproc.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n🔍 TIPOS DE DATOS:")
tipos_datos = {}
for dtype in df_preproc.dtypes:
    dtype_str = str(dtype)
    tipos_datos[dtype_str] = tipos_datos.get(dtype_str, 0) + 1
print(tipos_datos)

print("\n❌ VALORES NULOS:")
nulos = df_preproc.null_count()
nulos_dict = {}
for i, col in enumerate(df_preproc.columns):
    nulos_dict[col] = nulos[0, i]

nulos_positivos = {k: v for k, v in nulos_dict.items() if v > 0}
if nulos_positivos:
    # Ordenar por valores descendentes
    nulos_ordenados = sorted(nulos_positivos.items(), key=lambda x: x[1], reverse=True)
    for col, count in nulos_ordenados:
        print(f"{col}: {count}")
else:
    print("No hay valores nulos")


In [ ]:

print("\n📈 ESTADÍSTICAS BÁSICAS:")
df_preproc.describe()

In [ ]:
print("="*60)
print("ANÁLISIS DE CAMPOS CATEGÓRICOS CLAVE")
print("="*60)

campos_clave = [
    'ESTADO REGISTRO', 
    'ESTADO CUM', 
    'MUESTRA MÉDICA',
    'ATC',
    'VÍA ADMINISTRACIÓN',
    'PRINCIPIO ACTIVO',
    'FORMA FARMACÉUTICA'
]
for campo in campos_clave:
    print(f"\n{'='*80}")
    print(f"📊 ANÁLISIS DE: {campo}")
    print(f"{'='*80}")
    
    # Contar valores únicos y sus frecuencias, ordenado por count descendente
    conteos = df_preproc[campo].value_counts().sort("count", descending=True)
    total_registros = len(df_preproc)
    
    print(f"📈 Total de valores únicos: {len(conteos):,}")
    print(f"📋 Total de registros: {total_registros:,}")
    
    # Calcular porcentajes y mostrar top 10
    print(f"\n🏆 TOP 10 valores más frecuentes:")
    print("-" * 80)
    
    for i, fila in enumerate(conteos.head(10).iter_rows(named=True)):
        valor = fila[campo]
        conteo = fila['count']
        porcentaje = (conteo / total_registros) * 100
        print(f"{i+1:2d}. {valor:<45} | {conteo:>8,} ({porcentaje:>6.2f}%)")
    
    # Mostrar valores con menor frecuencia si hay más de 10
    if len(conteos) > 10:
        print(f"\n📉 Valores menos frecuentes (últimos 5):")
        print("-" * 80)
        tail_valores = conteos.tail(5)
        for i, fila in enumerate(tail_valores.iter_rows(named=True)):
            valor = fila[campo]
            conteo = fila['count']
            porcentaje = (conteo / total_registros) * 100
            print(f"   {valor:<45} | {conteo:>8,} ({porcentaje:>6.2f}%)")


In [ ]:
print("="*60)
print("ANÁLISIS VÁLIDOS VS INVÁLIDOS PARA HOMOLOGACIÓN")
print("="*60)

# Criterios para medicamentos VÁLIDOS (candidatos para homologación)
mascara_validos = (
    (df_preproc['ESTADO REGISTRO'] == 'Vigente') & 
    (df_preproc['ESTADO CUM'] == 'Activo') & 
    (df_preproc['MUESTRA MÉDICA'] == 'No')
)

# Medicamentos INVÁLIDOS (necesitan homologación)
mascara_invalidos = ~mascara_validos

validos = df_preproc.filter(mascara_validos)
invalidos = df_preproc.filter(mascara_invalidos)

print(f"📊 DISTRIBUCIÓN:")
print(f"   ✅ Medicamentos VÁLIDOS (candidatos): {len(validos):,} ({len(validos)/len(df_preproc)*100:.1f}%)")
print(f"   ❌ Medicamentos INVÁLIDOS (a homologar): {len(invalidos):,} ({len(invalidos)/len(df_preproc)*100:.1f}%)")

# Análisis de por qué son inválidos
print(f"\n🔍 RAZONES DE INVALIDEZ:")

no_vigente = df_preproc['ESTADO REGISTRO'] != 'Vigente'
no_activo = df_preproc['ESTADO CUM'] != 'Activo'
es_muestra = df_preproc['MUESTRA MÉDICA'] == 'Si'

print(f"   📋 Estado Registro ≠ 'Vigente': {no_vigente.sum():,}")
print(f"   📋 Estado CUM ≠ 'Activo': {no_activo.sum():,}")
print(f"   📋 Es Muestra Médica: {es_muestra.sum():,}")

# Análisis de solapamiento
print(f"\n📈 ANÁLISIS DE SOLAPAMIENTO:")
print(f"   Solo por Estado Registro: {(no_vigente & ~no_activo & ~es_muestra).sum():,}")
print(f"   Solo por Estado CUM: {(~no_vigente & no_activo & ~es_muestra).sum():,}")
print(f"   Solo por Muestra Médica: {(~no_vigente & ~no_activo & es_muestra).sum():,}")


In [ ]:
print("="*60)
print("ANÁLISIS DE PRINCIPIOS ACTIVOS")
print("="*60)

principios = df_preproc['PRINCIPIO ACTIVO'].value_counts().sort("count", descending=True)

print(f"🧬 Total de principios activos únicos: {len(principios):,}")
print(f"📊 Principios con un solo medicamento: {(principios['count'] == 1).sum():,}")
print(f"📊 Principios con múltiples medicamentos: {(principios['count'] > 1).sum():,}")

print(f"\n🔝 TOP 15 PRINCIPIOS MÁS FRECUENTES:")
for i, fila in enumerate(principios.head(15).iter_rows(named=True), 1):
    principio = fila['PRINCIPIO ACTIVO']
    conteo = fila['count']
    print(f"   {i:2d}. {principio}: {conteo:,} medicamentos")


In [ ]:
import matplotlib.pyplot as plt

print("="*60)
print("GENERANDO VISUALIZACIONES")
print("="*60)

# Configurar subplots
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Distribuciones Principales - Dataset Medicamentos', fontsize=16, fontweight='bold')

# 1. Estado Registro
estado_reg = df_preproc['ESTADO REGISTRO'].value_counts().sort("count", descending=True)
labels = estado_reg.select(pl.col('ESTADO REGISTRO')).to_series().to_list()
values = estado_reg.select(pl.col('count')).to_series().to_list()
axes[0,0].pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
axes[0,0].set_title('Estado Registro')

# 2. Estado CUM
estado_cum = df_preproc['ESTADO CUM'].value_counts().sort("count", descending=True)
labels = estado_cum.select(pl.col('ESTADO CUM')).to_series().to_list()
values = estado_cum.select(pl.col('count')).to_series().to_list()
axes[0,1].pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
axes[0,1].set_title('Estado CUM')

# 3. Muestra Médica
muestra = df_preproc['MUESTRA MÉDICA'].value_counts().sort("count", descending=True)
labels = muestra.select(pl.col('MUESTRA MÉDICA')).to_series().to_list()
values = muestra.select(pl.col('count')).to_series().to_list()
axes[0,2].pie(values, labels=labels, autopct='%1.1f%%', startangle=90)
axes[0,2].set_title('Muestra Médica')

# 4. Top 10 Vías de Administración
vias = df_preproc['VÍA ADMINISTRACIÓN'].value_counts().sort("count", descending=True).head(10)
labels = vias.select(pl.col('VÍA ADMINISTRACIÓN')).to_series().to_list()
values = vias.select(pl.col('count')).to_series().to_list()
axes[1,0].barh(range(len(values)), values)
axes[1,0].set_yticks(range(len(values)))
axes[1,0].set_yticklabels(labels)
axes[1,0].set_title('Top 10 Vías de Administración')
axes[1,0].set_xlabel('Frecuencia')

# 5. Top 10 Formas Farmacéuticas
formas = df_preproc['FORMA FARMACÉUTICA'].value_counts().sort("count", descending=True).head(10)
labels = formas.select(pl.col('FORMA FARMACÉUTICA')).to_series().to_list()
values = formas.select(pl.col('count')).to_series().to_list()
axes[1,1].barh(range(len(values)), values)
axes[1,1].set_yticks(range(len(values)))
axes[1,1].set_yticklabels(labels)
axes[1,1].set_title('Top 10 Formas Farmacéuticas')
axes[1,1].set_xlabel('Frecuencia')

# 6. Válidos vs Inválidos
mascara_validos = (
    (df_preproc['ESTADO REGISTRO'] == 'Vigente') & 
    (df_preproc['ESTADO CUM'] == 'Activo') & 
    (df_preproc['MUESTRA MÉDICA'] == 'No')
)
validos_invalidos = ['Válidos', 'Inválidos']
conteos = [mascara_validos.sum(), (~mascara_validos).sum()]

axes[1,2].pie(conteos, labels=validos_invalidos, autopct='%1.1f%%', 
                colors=['lightgreen', 'lightcoral'], startangle=90)
axes[1,2].set_title('Medicamentos Válidos vs Inválidos')

plt.tight_layout()
plt.show()